## Step 1: Setup and Load Data

First, configure your burst analysis and create the pipeline (but don't run it yet).

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import sys

# Add parent directory to path
sys.path.insert(0, str(Path.cwd().parent))

from scat_analysis.burstfit_pipeline import BurstPipeline
from scat_analysis.burstfit_interactive import InitialGuessWidget

# Configuration
burst_name = "freya"  # Change to your burst name
telescope = "chime"    # or "dsa"
data_path = Path("../plots/test/freya_synthetic_v2.npy")  # Your data file
plot_dir = Path("../plots/tutorial")
plot_dir.mkdir(parents=True, exist_ok=True)

# Pipeline parameters
pipeline_params = {
    'telescope': telescope,
    'telcfg_path': '../configs/telescopes.yaml',
    'sampcfg_path': '../configs/sampler.yaml',
    'steps': 500,  # Fewer steps for tutorial
    'f_factor': 384,
    't_factor': 2,
    'center_burst': True,
    'outer_trim': 0.49,
    'smooth_ms': 0.1,
    'nproc': 4,  # Adjust based on your system
    'yes': True
}

print(f"Configuration for {burst_name.upper()} ({telescope.upper()})")
print(f"Data: {data_path}")
print(f"Output: {plot_dir}")

## Step 2: Create Pipeline (Load Data)

This creates the pipeline object and loads/preprocesses your data, but **doesn't run MCMC yet**.

In [ ]:
# Create pipeline - this loads and preprocesses the data
pipe = BurstPipeline(
    name=burst_name,
    inpath=data_path,
    outpath=plot_dir,
    dm_init=0.0,  # DM correction (adjust as needed)
    **pipeline_params
)

print("✓ Pipeline created successfully")
print(f"  Data shape: {pipe.dataset.data.shape}")
print(f"  Time range: {pipe.dataset.time[0]:.3f} - {pipe.dataset.time[-1]:.3f} ms")
print(f"  Frequency range: {pipe.dataset.freq[0]:.3f} - {pipe.dataset.freq[-1]:.3f} GHz")

## Step 3: Launch Interactive Widget

This displays an interactive interface with:
- **Sliders** for all model parameters
- **Real-time visualization** of Data, Model, Residual, and Time Profile
- **Auto-Optimize button** to refine from current slider values
- **Accept & Continue button** to save parameters

### How to Use:

1. **Adjust sliders** to make the model roughly match the data
   - Look at the time profile plot - does the peak align?
   - Check the residual - is it mostly noise with no structure?
   - Watch the χ² value decrease as you improve the fit

2. **Click "Auto-Optimize"** to let scipy refine your guess
   - The sliders will update automatically
   - This works best when you're already close to the right answer

3. **Click "Accept & Continue"** when satisfied
   - Parameters are saved and ready to use

### Parameter Guide:

- **c0 (amplitude)**: Overall brightness of the burst
- **t0 (arrival)**: Time of burst peak (ms)
- **gamma (width)**: Pulse width parameter (log scale: -2 = narrow, 2 = wide)
- **zeta (spectral width)**: Frequency structure (0.1 = smooth, 5 = variable)
- **tau_1ghz (scatter)**: Scattering time at 1 GHz (ms) - look for exponential tail
- **alpha (scatter index)**: Frequency dependence of scattering (typically 3.5-4.5)

In [ ]:
# Create and display the interactive widget
guess_widget = InitialGuessWidget(
    dataset=pipe.dataset,
    model_key="M3"  # Full scattering model
)

# Display the widget
display(guess_widget.create_widget())

print("\n" + "="*70)
print("INTERACTIVE WIDGET INSTRUCTIONS")
print("="*70)
print("1. Adjust sliders to roughly match data (watch time profile!)")
print("2. Click 'Auto-Optimize' to refine with scipy")
print("3. Click 'Accept & Continue' when satisfied")
print("4. Run the next cell to inject parameters into pipeline")
print("="*70)

## Step 4: Inject Refined Parameters

After you've accepted the parameters in the widget, run this cell to inject them into the pipeline.

In [ ]:
# Get the refined parameters from the widget
optimized_guess = guess_widget.get_params()

# Inject into the pipeline
pipe.seed_single = optimized_guess

print("✓ Custom initial guess injected into pipeline")
print("\nRefined parameters:")
print("-" * 50)
for key, val in optimized_guess.__dict__.items():
    if val is not None and isinstance(val, (int, float)):
        print(f"  {key:12s}: {val:.4f}")

print("\nThe pipeline will now use these parameters as the starting point for MCMC.")

## Step 5: Run Pipeline with Refined Guess

Now run the full pipeline. It will use your refined initial guess instead of the automated one.

In [ ]:
# Run the full pipeline with your refined initial guess
print("Running pipeline with refined initial guess...")
print("(This may take a few minutes)")

results = pipe.run_full(
    model_scan=False,      # Skip model comparison for tutorial
    model_keys=["M3"],     # Just fit the full scattering model
    diagnostics=False,     # Skip extra diagnostics for speed
    plot=True,             # Generate plots
    save=True,             # Save results
    show=False             # Don't block with plot windows
)

print("\n" + "="*70)
print("PIPELINE COMPLETE!")
print("="*70)
print(f"Best model: {results['best_key']}")
if 'goodness_of_fit' in results:
    print(f"Reduced χ²: {results['goodness_of_fit']['chi2_reduced']:.2f}")

print("\nBest-fit parameters (posterior medians):")
print("-" * 50)
flat_chain = results['flat_chain']
param_names = results['param_names']
for i, name in enumerate(param_names):
    median = np.median(flat_chain[:, i])
    std = np.std(flat_chain[:, i])
    print(f"  {name:12s}: {median:.4f} ± {std:.4f}")

## Step 6: Compare Initial Guess vs Final Result

Let's visualize how your initial guess compares to the final MCMC result.

In [ ]:
from scat_analysis.burstfit import FRBModel

# Get initial guess and final parameters
initial_params = optimized_guess
final_params = results['best_params']

# Generate models
model_instance = results['model_instance']
model_initial = model_instance(initial_params, "M3")
model_final = model_instance(final_params, "M3")

# Plot comparison
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

# Data
vmin, vmax = np.percentile(pipe.dataset.data, [1, 99])
im0 = axes[0,0].imshow(pipe.dataset.data, aspect='auto', origin='lower',
                       extent=[pipe.dataset.time[0], pipe.dataset.time[-1],
                              pipe.dataset.freq[0], pipe.dataset.freq[-1]],
                       vmin=vmin, vmax=vmax, cmap='plasma')
axes[0,0].set_title('Observed Data', fontweight='bold', fontsize=12)
axes[0,0].set_ylabel('Frequency [GHz]')
plt.colorbar(im0, ax=axes[0,0])

# Initial guess model
im1 = axes[0,1].imshow(model_initial, aspect='auto', origin='lower',
                       extent=[pipe.dataset.time[0], pipe.dataset.time[-1],
                              pipe.dataset.freq[0], pipe.dataset.freq[-1]],
                       vmin=vmin, vmax=vmax, cmap='plasma')
axes[0,1].set_title('Initial Guess (Your Refinement)', fontweight='bold', fontsize=12)
plt.colorbar(im1, ax=axes[0,1])

# Final MCMC model
im2 = axes[0,2].imshow(model_final, aspect='auto', origin='lower',
                       extent=[pipe.dataset.time[0], pipe.dataset.time[-1],
                              pipe.dataset.freq[0], pipe.dataset.freq[-1]],
                       vmin=vmin, vmax=vmax, cmap='plasma')
axes[0,2].set_title('Final MCMC Result', fontweight='bold', fontsize=12)
plt.colorbar(im2, ax=axes[0,2])

# Time profiles
time_data = np.sum(pipe.dataset.data, axis=0)
time_initial = np.sum(model_initial, axis=0)
time_final = np.sum(model_final, axis=0)

axes[1,0].plot(pipe.dataset.time, time_data, 'k-', lw=1.5, alpha=0.7, label='Data')
axes[1,0].plot(pipe.dataset.time, time_initial, 'b-', lw=2, label='Initial Guess')
axes[1,0].set_xlabel('Time [ms]')
axes[1,0].set_ylabel('Intensity')
axes[1,0].set_title('Initial Guess Comparison', fontweight='bold')
axes[1,0].legend()
axes[1,0].grid(alpha=0.3)

axes[1,1].plot(pipe.dataset.time, time_data, 'k-', lw=1.5, alpha=0.7, label='Data')
axes[1,1].plot(pipe.dataset.time, time_final, 'm-', lw=2, label='Final MCMC')
axes[1,1].set_xlabel('Time [ms]')
axes[1,1].set_ylabel('Intensity')
axes[1,1].set_title('Final MCMC Comparison', fontweight='bold')
axes[1,1].legend()
axes[1,1].grid(alpha=0.3)

# Parameter comparison
axes[1,2].axis('off')
comparison_text = "Parameter Evolution:\n" + "="*30 + "\n\n"
for key in ['c0', 't0', 'gamma', 'zeta', 'tau_1ghz', 'alpha']:
    initial_val = getattr(initial_params, key, None)
    final_val = getattr(final_params, key, None)
    if initial_val is not None and final_val is not None:
        change = ((final_val - initial_val) / initial_val * 100) if initial_val != 0 else 0
        comparison_text += f"{key:12s}:\n"
        comparison_text += f"  Initial: {initial_val:8.4f}\n"
        comparison_text += f"  Final:   {final_val:8.4f}\n"
        comparison_text += f"  Change:  {change:+6.1f}%\n\n"

axes[1,2].text(0.1, 0.9, comparison_text, 
              transform=axes[1,2].transAxes,
              fontfamily='monospace', fontsize=9,
              verticalalignment='top')
axes[1,2].set_title('Parameter Refinement', fontweight='bold')

plt.tight_layout()
plt.savefig(plot_dir / 'initial_vs_final_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"\n✓ Comparison plot saved to: {plot_dir / 'initial_vs_final_comparison.png'}")

## Summary

You've successfully used the interactive widget to:

1. ✓ Load and visualize FRB data
2. ✓ Manually refine initial parameter guess with real-time feedback
3. ✓ Auto-optimize from your manual starting point
4. ✓ Run MCMC starting from a good initial position
5. ✓ Compare your initial guess to the final MCMC result

### Key Takeaways:

- **The widget helps you avoid local minima** by letting you visually guide the fit
- **Auto-optimize works better** when you start from a reasonable position
- **MCMC converges faster** with a good initial guess
- **Your manual refinement provides intuition** about the parameter space

### When to Use This Workflow:

- ✓ When automated initial guess is clearly wrong
- ✓ For bursts with unusual structure (multiple components, weak scattering, etc.)
- ✓ When MCMC keeps failing or giving unreasonable results
- ✓ When you want to understand parameter sensitivities

### Next Steps:

Try running this tutorial with different bursts from your dataset. You'll develop intuition for:
- How each parameter affects the model
- What "good" vs "bad" residuals look like
- When manual intervention is needed vs automated guess is fine